In [31]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [32]:
# 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows의 경우 '맑은 고딕'
# Mac의 경우: plt.rcParams['font.family'] = 'AppleGothic'
# Linux의 경우 적절한 한글 폰트를 설치해야 함 (예: 'NanumGothic')

# 음수 기호가 깨지는 문제 해결 (minus 폰트 설정)
plt.rcParams['axes.unicode_minus'] = False

In [33]:
# 두 개의 Excel 파일을 읽습니다.
df1 = pd.read_excel('pre_input_303F.xlsx')
df2 = pd.read_excel('pre_defect_303F.xlsx')

In [34]:
# Convert string column to year-month format
def convert_to_year_month(value):
    if isinstance(value, int):
        year = str(value)[:4]
        month = str(value)[4:]
        return f'{year}-{month.zfill(2)}'
    elif isinstance(value, str):
        year = value[:4]
        month = value[4:]
        return f'{year}-{month.zfill(2)}'
    else:
        return value

df1['년월'] = df1['년월'].apply(convert_to_year_month)

def convert_to_Rolling_year_month_date(date):
    year = date.year
    month = date.month
    return f'{year}-{month:02}' 


df1['압연년월'] = df1['최종압연일자'].apply(convert_to_Rolling_year_month_date)

# 압연년월을 datetime으로 변환 후 연도-월만 추출
df1['압연년월'] = pd.to_datetime(df1['압연년월']).dt.strftime('%Y-%m')

# 압연년월+heat_no
df1['압연년월_Heat'] = df1['압연년월']+'_'+df1['HEAT_NO'].astype(str)

df1.head()

,압연년,압연월,년월,HEAT_NO,LOT_NO,공정코드,구분,공장명,주조구분,재료코드,...,주여구분,지시GR종류,생산GR종류,생산GR공정,제강검사완료일,작업일,강관소재여부,주설비구분,압연년월,압연년월_Heat
0,2023,8,2023-08,S54745,B360553701,9Y01,압연,산세,T,7730.0,...,1,A,A,8301,2023-06-27,2023-10-04,N,C,2023-08,2023-08_S54745
1,2023,8,2023-08,S54745,B360553901,9Y01,압연,산세,T,7730.0,...,1,A,A,8301,2023-06-27,2023-10-04,N,C,2023-08,2023-08_S54745
2,2023,2,2023-02,S52717,B310559703,9Y01,압연,산세,T,7730.0,...,1,A,A,8301,2023-01-04,2023-02-20,N,W,2023-02,2023-02_S52717
3,2023,2,2023-02,S52717,B310559702,9Y01,압연,산세,T,7730.0,...,1,A,A,8301,2023-01-04,2023-02-20,N,W,2023-02,2023-02_S52717
4,2023,2,2023-02,S52717,B310559701,9Y01,압연,산세,T,7730.0,...,1,A,A,8301,2023-01-04,2023-02-20,N,W,2023-02,2023-02_S52717


In [35]:
def convert_to_year_month_date(date):
    year, month, _ = date.split('-')
    return f'{year}-{month}'

def convert_to_Rolling_year_month_date(date):
    year = date.year
    month = date.month
    return f'{year}-{month:02}' 

df2['년월'] = df2['년월일'].apply(convert_to_year_month_date)
df2['압연년월'] = df2['최종압연일자'].apply(convert_to_Rolling_year_month_date)

# 압연년월을 datetime으로 변환 후 연도-월만 추출
df2['압연년월'] = pd.to_datetime(df2['압연년월']).dt.strftime('%Y-%m')

# 압연년월+heat_no
df2['압연년월_Heat'] = df2['압연년월']+'_'+df2['HEAT_NO'].astype(str)

# 년월은 년월일 컬럼 앞에 배치
position = df2.columns.get_loc('년월일')
df2.insert(position, '년월', df2.pop('년월'))

df2.head()

,압연년,압연월,년월,년월일,업무구분,공장구분,HEAT_NO,LOT_NO,PON,공정순위,...,압연폭4,최종압연공정,최종압연일자,제품압연작업조,생성일시,HCR,투입중량,주설비구분,압연년월,압연년월_Heat
0,2024,4,2024-04,2024-04-29,2,9,S55761,B3A0481500,D24W3W1444,100,...,0,8100,2024-04-22,B,2024-05-03,NaN,NaN,C,2024-04,2024-04_S55761
1,2024,4,2024-04,2024-04-29,2,9,S55761,B3A0481500,D24W3W1444,100,...,0,8100,2024-04-22,B,2024-05-03,NaN,NaN,C,2024-04,2024-04_S55761
2,2023,8,2023-09,2023-09-11,2,9,S54745,B360553802,D23W8W1198,120,...,0,8100,2023-08-28,C,2023-10-05,NaN,NaN,C,2023-08,2023-08_S54745
3,2023,10,2023-10,2023-10-26,2,9,S55018,B370540802,D23WAW0353,90,...,0,8100,2023-10-11,C,2023-11-02,NaN,NaN,W,2023-10,2023-10_S55018
4,2023,10,2023-10,2023-10-26,2,9,S55018,B370540802,D23WAW0353,90,...,0,8100,2023-10-11,C,2023-11-02,NaN,NaN,W,2023-10,2023-10_S55018


In [36]:
# 데이터 병합
df = pd.concat([df1, df2], ignore_index=True)

# 병합된 데이터를 새로운 파일로 저장
df.to_excel('df_b.xlsx', index=False)

In [37]:
# '압연외경'으로 그룹화하여 년월 기준으로 계산
df['압연년월'] = pd.to_datetime(df['압연년월']).dt.strftime('%Y-%m')  # '날짜' 열에서 년월을 문자열로 변환
grouped = df.groupby(['외경', '압연년월']).agg({
    '투입중량': 'sum',
    '발생중량': 'sum'
}).reset_index()

# 불량율 계산
grouped['불량율'] = (grouped['발생중량'] / grouped['투입중량']) * 100

# 고유한 '외경' 값 가져오기
unique_conditions = grouped['외경'].unique()

# 그래프를 저장할 폴더 생성
output_folder = '압연월_외경별 불량율'  # 사용자에게 폴더 경로 입력 요청
os.makedirs(output_folder, exist_ok=True)  # 폴더가 없으면 생성

# 각 필터 조건에 대해 그래프 생성
for condition in unique_conditions:
    filtered_df = grouped[grouped['외경'] == condition]  # 특정 조건으로 필터링

    if not filtered_df.empty:  # 필터링된 데이터가 있는 경우에만 그래프 그리기

        # '압연년월'을 문자열로 변환하여 정렬
        filtered_df = filtered_df.sort_values(by='압연년월')  # x축 오름차순 정렬

        plt.figure(figsize=(12, 6))
        
        # Seaborn으로 불량율 그래프 그리기
        sns.lineplot(x='압연년월', y='불량율', data=filtered_df, marker='o')

        # 그래프 제목 및 축 레이블 설정
        plt.xticks(rotation=90)
        plt.title(f'불량율 by 압연년월 for 외경 {condition}')
        plt.xlabel('압연년월')
        plt.ylabel('불량율')

        # 각 데이터 포인트에 값 표시
        for index, row in filtered_df.iterrows():
            plt.text(row['압연년월'], row['불량율'], f"{row['불량율']:.1f}", 
                     color='black', ha='center', va='bottom')

        # 레이아웃 조정 (그래프에서 데이터가 잘리지 않도록)
        plt.tight_layout()

        # 그래프 저장 (조건에 따라 파일 이름 설정)
        plt.savefig(os.path.join(output_folder, f'불량율_외경_{condition}.png'), dpi=300)  # dpi는 해상도

        plt.close()  # 그래프를 닫아 메모리 관리

# 완료 메시지
print("모든 그래프가 저장되었습니다.")

posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


모든 그래프가 저장되었습니다.


In [38]:
# '압연외경'으로 그룹화하여 년월 기준으로 계산
df['압연년월'] = pd.to_datetime(df['압연년월']).dt.strftime('%Y-%m')  # '날짜' 열에서 년월을 문자열로 변환
grouped = df.groupby(['외경', '압연년월_Heat']).agg({
    '투입중량': 'sum',
    '발생중량': 'sum'
}).reset_index()

# 불량율 계산
grouped['불량율'] = (grouped['발생중량'] / grouped['투입중량']) * 100

# 고유한 '외경' 값 가져오기
unique_conditions = grouped['외경'].unique()

# 그래프를 저장할 폴더 생성
output_folder = '압연월_Heat별_외경별 불량율'  # 사용자에게 폴더 경로 입력 요청
os.makedirs(output_folder, exist_ok=True)  # 폴더가 없으면 생성

# 각 필터 조건에 대해 그래프 생성
for condition in unique_conditions:
    filtered_df = grouped[grouped['외경'] == condition]  # 특정 조건으로 필터링

    if not filtered_df.empty:  # 필터링된 데이터가 있는 경우에만 그래프 그리기

        # '압연년월'을 문자열로 변환하여 정렬
        filtered_df = filtered_df.sort_values(by='압연년월_Heat')  # x축 오름차순 정렬

        plt.figure(figsize=(12, 6))
        
        # Seaborn으로 불량율 그래프 그리기
        sns.lineplot(x='압연년월_Heat', y='불량율', data=filtered_df, marker='o')

        # 그래프 제목 및 축 레이블 설정
        plt.xticks(rotation=90)
        plt.title(f'불량율 by 압연년월_Heat for 외경 {condition}')
        plt.xlabel('압연년월_Heat')
        plt.ylabel('불량율')

        # 각 데이터 포인트에 값 표시
        for index, row in filtered_df.iterrows():
            plt.text(row['압연년월_Heat'], row['불량율'], f"{row['불량율']:.1f}", 
                     color='black', ha='center', va='bottom',rotation=0) # rotation=90 추가


        # 레이아웃 조정 (그래프에서 데이터가 잘리지 않도록)
        plt.tight_layout()

        # 그래프 저장 (조건에 따라 파일 이름 설정)
        plt.savefig(os.path.join(output_folder, f'불량율_외경_{condition}.png'), dpi=300)  # dpi는 해상도

        plt.close()  # 그래프를 닫아 메모리 관리

# 완료 메시지
print("모든 그래프가 저장되었습니다.")

posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


모든 그래프가 저장되었습니다.
